In [69]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split

In [70]:
with open("../data/others/db") as file:
    db = file.readlines()

db = [json.loads(line.removesuffix("\n")) for line in db]

In [71]:
df_db = pd.DataFrame.from_dict(db)
df_exploded = pd.concat([df_db, df_db["metadata"].apply(pd.Series)], axis=1)

In [72]:
def convert_row(row):
    return {elem["trait_type"]: elem["value"] for elem in row}


attributes = df_exploded["attributes"].apply(convert_row)
attributes = pd.DataFrame.from_dict(attributes.tolist())

In [73]:
df_final = pd.concat([df_exploded, attributes], axis=1)
df_final["image"] = df_final["image"].str.removeprefix("ipfs://")
df_final = df_final.loc[:, ["image", "Mouth", "Background", "Hat", "Eyes", "Clothes", "Fur", "Earring"]]
df_final = df_final.fillna("Null")

In [74]:
train_ids, rest_ids = train_test_split(df_final["image"], test_size=0.3, random_state=42)
validation_ids, test_ids = train_test_split(rest_ids, test_size=0.5, random_state=42)

In [75]:
df_final.loc[df_final["image"].isin(train_ids), "dataset"] = "train"
df_final.loc[df_final["image"].isin(validation_ids), "dataset"] = "validation"
df_final.loc[df_final["image"].isin(test_ids), "dataset"] = "test"

In [77]:
df_final.to_csv("../data/others/apes_info.csv", index=False)